## Bert - POC

In [2]:
import numpy as np
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

import os
import logging
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
class BertBaseMultilingualEmbeddingApi:
    
    def __init__(self, model_name="bert-base-multilingual-cased", cuda=True):
        self.device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name).to(self.device)
        self.model.eval()
        
        self.tokens_tensor = None
        self.segments_tensor = None
        
        self.encoded_layers_ = None
        self.token_embeddings_ = None
        
    def _tokenize_text(self, text):
        marked_text = "[CLS] " + text + " [SEP]"
        tokenized_text = self.tokenizer.tokenize(marked_text)
        indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
        segments_ids = [1] * len(tokenized_text)
        
        self.tokens_tensor = torch.tensor([indexed_tokens]).to(self.device)
        self.segments_tensor = torch.tensor([segments_ids]).to(self.device)
    
    def _evaluate(self):
        with torch.no_grad():
            encoded_layers, _ = self.model(self.tokens_tensor, self.segments_tensor)
        self.encoded_layers_ = encoded_layers
    
    def _generate_token_embeddings(self, batch_i=0):
        """
        Convert the hidden state embeddings into single token vectors
        Holds the list of 12 layer embeddings for each token
        Will have the shape: [# tokens, # layers, # features]
        """
        token_embeddings = [] 
        # For each token in the sentence...
        for token_i in range(len(self.encoded_layers_[-1][batch_i])):
            # Holds 12 layers of hidden states for each token 
            hidden_layers = [] 
            # For each of the 12 layers...
            for layer_i in range(len(self.encoded_layers_)):
            # Lookup the vector for `token_i` in `layer_i`
                vec = self.encoded_layers_[layer_i][batch_i][token_i]
                hidden_layers.append(vec)
            token_embeddings.append(hidden_layers)
        self.token_embeddings_ = token_embeddings
    
    def feed_forward(self, sentence):
        self._tokenize_text(sentence)
        self._evaluate()
        self._generate_token_embeddings()
    
    def create_word_embedding_(self, how="cat_last_4"):
        if how == "cat_last_4":
            return [torch.cat((layer[-1], layer[-2], layer[-3], layer[-4]), 0) for layer in self.token_embeddings_]
        elif how == "sum_last_4":
            return [torch.sum(torch.stack(layer)[-4:], 0) for layer in self.token_embeddings_]
        else:
            print("Redefine `how` parameter")
        
    def create_sentence_embedding_(self, how="mean_last_layer"):
        if how == "mean_last_layer":
            return torch.mean(self.encoded_layers_[-1], 1).squeeze()
        else:
            print("Redefine `how` parameter")
            
    def print_dimensions_(self):
        print ("Number of layers:", len(self.encoded_layers_))
        layer_i = 0
        print ("Number of batches:", len(self.encoded_layers_[layer_i]))
        batch_i = 0
        print ("Number of tokens:", len(self.encoded_layers_[layer_i][batch_i]))
        token_i = 0
        print ("Number of hidden units:", len(self.encoded_layers_[layer_i][batch_i][token_i]))
        
    def plot_embedding_hist(self, vec):
        sns.set_style("whitegrid")
        plt.figure(figsize=(12, 8))
        sns.distplot(vec)
        plt.show()

## Text Preprocessing

In [89]:
import codecs
from pprint import pprint
NUMBER_OF_FILES_TO_OPEN = 2000
CORPUS = "gazeta"

In [95]:
corpusNews = list()
for i in range(0,NUMBER_OF_FILES_TO_OPEN):
    try:
        f = codecs.open("doc/"+ CORPUS +"/doc"+str(i+1), "r", encoding = 'utf-8')
        lines = ""
        for line in f:
            lines += line
        corpusNews.append(lines)
        f.close()
    except FileNotFoundError:
        with open("doc/"+ CORPUS +"/doc"+str(i+1), "w", encoding = 'utf-8') as missing:
            pass
        corpusNews.append("")
        
pprint(corpusNews[0])
print(len(corpusNews))

('Tomasza <Entity name="Tomasz Sekielski" type="person" '
 'category="dziennikarze">Sekielskiego</Entity> oburzyła wypowiedź prof. '
 'Mariana Filara. Wybitny prawnik stwierdził, że nawet jeśli znalezione w celi '
 'Mariusza T. to prowokacja - postąpiono słusznie. - Nie wiem, co się stało '
 'profesorowi. Proponuje bardzo niebezpieczną drogę. Mariusz T. nigdy nie '
 'powinien wyjść na wolność. Ale nie można w tym celu łamać prawa - mówił '
 '<Entity name="Tomasz Sekielski" type="person" '
 'category="dziennikarze">Sekielski</Entity> w TOK FM. "Człowiek, który '
 'zdecydował o podrzuceniu Mariuszowi T. do celi kompromitujących materiałów, '
 'postawił na szali jedne wartości przeciw drugim. Postawił naruszenie prawa '
 'przeciw ludzkiemu bezpieczeństwu. W mojej ocenie wybrał słusznie" - tak w '
 'wywiadzie dla "Gazety Wyborczej" prof. <Entity name="Marian Filar" '
 'type="person" category="muzycy">Marian Filar</Entity> komentował informacje '
 'o znalezieniu w celi Mariusza T. materiałó

Below are functions from Piter, that I did not changed (but use it differently):

In [40]:
def find_annotations(document : str):
    '''
    Searches for all occurances of '<' and '>' in the document.
    Returns lists of indexes of occurances opening for '<' and closing for '>'
    '''
    i = 0
    opening = list()
    closing = list()
    while i != -1:
        i = document.find('<', i)
        opening.append(i)
        if i == -1:
            closing.append(-1)
            break
        i = document.find('>', i)
        closing.append(i)
    closing = [cl + 1 for cl in closing]
    closing[-1] = -1
    return(opening, closing)

def get_annotation_values(text : str): # jest typ nie type, bo type jest zarezerwowana nazwa, nie jestem uposledzony
    '''
    Returns a dict consisting annotation values {'name', 'typ', 'category'} for first occuring annotation
    in the text.
    '''
    name_start = text.find('name=') + len('name=\"')
    name_end = text.find('\"', name_start)
    typ_start = text.find('type=', name_end) + len('type=\"')
    typ_end = text.find('\"', typ_start)
    category_start = text.find('category', typ_end) + len('category=\"')
    category_end = text.find('\"', category_start)
    return({'name' : text[name_start:name_end], 'typ' : text[typ_start:typ_end],
               'category' : text[category_start:category_end]})

def exclude_vectors_for_person(list_of_vectors, person):
    '''
    list_of_vectors - return value from exclude_sentence_vectors()
    person - name of person from people_dict
    '''
    l = list()
    for el in list_of_vectors:
        if el[1] == person:
            l.append(el[0])
    return l

And our function for sentences:

In [88]:
def exclude_sentence_vectors(text):
    people_dict = {}
    sentences = text.split(".")
    contexts = []
    for sentence in sentences:
        opn, cls = find_annotations(sentence)
        if opn[0] != -1:
            #print(opn, cls)
            annotation = get_annotation_values(sentence)
            sentence = sentence[0:opn[0]] + sentence[cls[-2]:]
            contexts.append([sentence, annotation["name"]])
            try:
                people_dict[annotation["category"]].add(annotation["name"])
            except KeyError:
                people_dict[annotation["category"]] = {annotation["name"]}
    return contexts, people_dict


## vectors, people_dict = exclude_sentence_vectors(corpusNews[0])
print(vectors)
print(people_dict)

In [43]:
print(exclude_vectors_for_person(vectors, "Tomasz Sekielski"))

['Tomasza  oburzyła wypowiedź prof', ' Ale nie można w tym celu łamać prawa - mówił  w TOK FM', ' Słowa prawnika zaskoczyły i oburzyły ', ' Zdaniem  prawnik z UMK w Toruniu proponuje "bardzo niebezpieczną drogę"', ' - Ale nie można w tym celu łamać prawa, bo za chwilę w podobnej sytuacji może się znaleźć nie tylko morderca-pedofil - powiedział ']


## Generating tsv files

In [62]:
NAME = "name"
EMBEDDING = "embedding"
PROFESSION = "profession"


In [63]:
def generate_embeddings(news_file):
    embeddings = []
    vectors, people_dict = exclude_sentence_vectors(news_file)
    for profession in people_dict:
        names = people_dict[profession]
        for name in names:
            person_contexts = exclude_vectors_for_person(vectors, name)
            for context in person_contexts:
                bert.feed_forward(context)
                sent_embedding = bert.create_sentence_embedding_()
                embeddings.append({EMBEDDING:sent_embedding, NAME: name, PROFESSION: profession})
    return embeddings
        

In [64]:
def generate_tsv_files(filename, embeddings):
    with open(filename + "_vectors.tsv",'w') as f:
        for vector in embeddings:
            for value in vector[EMBEDDING]:
                f.write("%e\t" % (value))
            f.write("\n")
    f.close()

    with open(filename + "_metadata.tsv",'w') as f:
        f.write("Name\tProfession\n")
        for vector in embeddings:
            f.write("%s\t%s\n" % (vector[NAME].encode('utf-8'), vector[PROFESSION].encode('utf-8')))
    f.close()

In [96]:
TYPE = "mean_last_layer_"
for i in range(1690, NUMBER_OF_FILES_TO_OPEN):
    pprint(corpusNews[i])
    print(i)
    embeddings = generate_embeddings(corpusNews[i])
    generate_tsv_files("tsv_files/"+ CORPUS +"/" + TYPE+ "doc"+str(i+1), embeddings)

('Do częstochowskiego sądu trafił akt oskarżenia ws. brutalnego pobicia '
 '43-letniego pracownika firmy transportowej z Katowic. Mężczyzna został '
 'skatowany na polecenie właścicielki firmy, która miała zastrzeżenia do jego '
 'pracy. Doznał bardzo poważnych obrażeń. Informację o zakończeniu śledztwa w '
 'tej sprawie przekazał rzecznik Prokuratury Okręgowej w Częstochowie Tomasz '
 'Ozimek. Głównymi oskarżonymi są sprawca pobicia Patryk Z. i właścicielka '
 'firmy Anita N. We wrześniu ubiegłego roku firmy Anita N. zaczęła mieć '
 'zastrzeżenia do wykonywania przez niego obowiązków służbowych i '
 'przestrzegania dyscypliny pracy. 30 października przyjechała do Częstochowy '
 'w towarzystwie znajomych - Patryka Z. i Krzysztofa K. Miała odebrać od '
 'kierowcy samochód służbowy. Jak poinformowała prokuratura, cała trójka '
 'wtargnęła do mieszkania, w którym Jacek K. mieszka wraz z matką, żądając '
 'wydania kluczyków oraz dokumentów samochodu. Na samych żądaniach się nie '
 'skończy

('Siły bezpieczeństwa południowej Rosji zostały postawione w stan alarmu '
 'bojowego po tym, jak w Stawropolu znaleziono pięć ciał z ranami '
 'postrzałowymi - donosi Reuters. Inne źródła mówią o sześciu ciałach. Lokalne '
 'władze ogłosiły tam "reżim operacji antyterrorystycznej", będący rodzajem '
 'stanu wyjątkowego. W dwóch rejonach Kraju Stawropolskiego, na południu '
 'Rosji, wprowadzono w czwartek "reżim operacji antyterrorystycznej", czyli '
 'rodzaj stanu wyjątkowego. Decyzję tę podjęto po znalezieniu tam czterech '
 'samochodów osobowych z ciałami pięciu zastrzelonych mężczyzn. Ciała '
 'znajdowały się dwóch różnych dystryktach położonych niedaleko Stawropolu. W '
 'jednym z aut odkryto też "niezidentyfikowany ładunek wybuchowy" - '
 'poinformował rzecznik Federalnej Służby Bezpieczeństwa. Miasto to jest '
 'określane mianem "bramy na Kaukaz" i uważane przez władze rosyjskie za punkt '
 'newralgiczny. Kraj Stawropolski graniczy m.in. z Krajem Krasnodarskim, na '
 'terytorium

('50-letnia mieszkanka powiatu elbląskiego padła ofiarą oszusta\xa0'
 'matrymonialnego, którego poznała na jednym z internetowych\xa0portali '
 'randkowych. - Mężczyzna podawał się za amerykańskiego generała. Pisał\xa0, '
 'że jest obecnie w Afganistanie. Obiecywał, że przyjedzie do Polski, a co '
 'więcej przywiezie skarb, który znalazł będąc na\xa0misji - opowiada <Entity '
 'name="Krzysztof Nowacki" type="person" category="sportowcy">Krzysztof '
 'Nowacki</Entity>, rzecznik elbląskiej policji. - Kobieta uwierzyła we '
 'wszystko co jej pisał. Dała się uwieść i przekazała\xa0oszustowi ponad 200 '
 'tysięcy złotych. Policjanci przestrzegają przed tzw. "oszustwem na '
 'żołnierza". - Uwaga! Ofiarą sprawców takich czynów mogą paść również '
 'mężczyźni. Znany jest bowiem przypadek, gdy kobieta podająca się za oficerów '
 'wojska na zagranicznej misji, oszukała poznanego przez Internet mężczyznę. '
 'Sprawcy tego specyficznego oszustwa działają na dwóch płaszczyznach najpierw '
 'zdobywa

('Po kilku miesiącach ujawniono film z katastrofy samolotu na Islandii. W '
 'sierpniu w Akureyri zginęły dwie osoby, ale drugi pilot przeżył tę '
 'dramatycznie wyglądającą katastrofę. Na nagraniu widać samolot Beechcraft '
 'King Air 200 pogotowia lotniczego, który uderza w ziemię i wybucha. Śledztwo '
 'w tej sprawie nadal trwa. Choć trudno w to uwierzyć, oglądając nagranie, z '
 'trzech osób znajdujących się na pokładzie - jedna przeżyła katastrofę. To '
 'drugi pilot maszyny. Jak twierdzi użytkownik, który zamieścił na LiveLeak, '
 'przeżył on już jedno awaryjne lądowanie, 12 lat temu.')
1326
('W Kędzierzawicach pod Warszawą policjanci odkopali ciało noworodka. Na razie '
 'nie wiadomo, co było przyczyną śmierci dziecka Zatrzymana już przez policję '
 '40-letnia matka dziecka ma jeszcze czwórkę dzieci. Według nieoficjalnych '
 'informacji rodzina jest pod opieką lokalnego ośrodka pomocy społecznej. Jego '
 'pracownicy w ub. roku dowiedzieli się, że kobieta jest w ciąży. W '
 'paźd

('Blisko ćwierć miliona złotych prezydent Gdyni Wojciech Szczurek zapłaci '
 'indyjskiej spółce WNS za promocję. W witrynie WNS Services umieszczony '
 'został link do miejskiej strony internetowej. To promocja miasta czy forma '
 'wsparcia dla inwestora, który otworzył biura w mieście? Na promocję Urząd '
 'Miasta Gdyni wyda w przyszłym roku ok. 15 mln zł i o większości podjętych '
 'działań będzie głośno. Tymczasem bez większego rozgłosu Gdynia rozstrzygnęła '
 'postępowanie z wolnej ręki na promocję miasta przez spółkę WNS Global '
 'Services. To koncern z Indii, który przez oddział w Londynie otworzył w '
 'Gdyni biura. WNS świadczy usługi outsourcingowe (finansowo-księgowe, prawne, '
 'sprzedażowe, analityczne) dla wielu korporacji. Ma swoje oddziały m.in. w '
 'Indiach, Bukareszcie, Indiach, Kostaryce, RPA, na Filipinach i Sri Lance. Za '
 'co 246 tys.? Z dokumentów wynika, że Gdynia zapłaci koncernowi 246 tys. zł '
 'za "umieszczenie na stronie internetowej firmy wns.com linku d

...

('Justine Sacco, dyrektor komunikacji i PR koncernu InterActiveCorp, została '
 'zwolniona z pracy. Wszystko przez tweeta, którego wysłała przed podróżą do '
 'Kapsztadu: "Jadę do Afryki. Mam nadzieję, że nie złapię AIDS. Och, '
 'żartowałam. Jestem biała!". Teraz Sacco przeprasza mieszkańców RPA. "Mój '
 'ojciec pochodzi z RPA. Ja też się tam urodziłam" - napisała w oświadczeniu. '
 'Przez kilkanaście godzin lotu jej wpis wywołał ogromny skandal w internecie. '
 'Został skrytykowany przez tysiące osób, a sama firma - nowojorska '
 'InterActiveCorp, której własnością są takie internetowe marki, jak: Daily '
 'Beast, Vimeo, About.com, Match.com i Ask.com, wydała oświadczenie, w którym '
 'pisała: "To szokujący, obraźliwy komentarz, który nie odzwierciedla poglądów '
 'i wartości IAC. To bardzo poważna sprawa i podejmujemy odpowiednie kroki". '
 'Jak dodała firma, "niestety osoba, o którą chodzi, jest niedostępna, leci '
 'międzynarodowym lotem". Osoby na tak wysokim stanowisku nie powin

('- Proponujemy księżom, by nie czytali waszego listu w ostatnią niedzielę '
 'tego roku. Chcemy w tym dniu posłuchać o Zbawicielu, Bogu i Ewangelii - '
 'pisze Błażej Strzelczyk, dziennikarz i twórca bloga #FollowJesus. Odpowiada '
 'w ten sposób na list polskich biskupów, w którym m.in. straszą oni osławioną '
 'już "ideologią gender". List, w którym biskupi ostrzegają m.in. przed '
 'złowrogą "gender, która zagraża rodzinie" ma być odczytany w niedzielę w '
 'kościołach. Nie o tym chcemy słuchać na mszy - podkreśla dziennikarz '
 '"Tygodnika Powszechnego" Błażej Strzelczyk. Jego list do biskupów szybko '
 'zyskał dużą popularność w sieci. "Drodzy Księża Biskupi, właśnie '
 'przeczytałem Wasz list z dalekiego Episkopatu, za który bardzo Wam dziękuję. '
 'Macie tam dużo problemów, wielu wrogów, doszły mnie również słuchy, że '
 'przygotowujecie się na akcję masowych aresztowań. Bardzo się za Was modlimy. '
 'Pozwólcie więc na garść słów otuchy w tym trudnym dla Was czasie. U nas - '
 

('W przemówieniu do kardynałów i biskupów z Kurii Rzymskiej papież Franciszek '
 'powiedział dziś, że jeśli zabraknie tam postawy służby Kościołowi, będzie to '
 '"ciążący urząd biurokratyczny; kontrolny i inkwizycyjny". Apelował o '
 '"sprzeciw sumienia wobec plotek". Tradycyjne spotkania z Kurią Rzymską to '
 'okazja do złożenia świątecznych życzeń. Poprzednicy Franciszka - Benedykt '
 'XVI i Jan Paweł II - w wygłaszanych wtedy przemówieniach dokonywali '
 'podsumowania roku w Kościele. Papież skoncentrował się w swym wystąpieniu na '
 'samej Kurii Rzymskiej - rządzie Kościoła, który zamierza gruntownie '
 'zreformować. Zwracając się do pracowników Kurii, Franciszek powiedział: '
 '"Podczas tych moich pierwszych świąt Bożego Narodzenia jako Biskup Rzymu, '
 'odczuwam potrzebę, by powiedzieć wielkie dziękuję". - Dziękuję wam za waszą '
 'codzienną posługę: za troskę, pracowitość, kreatywność; za nie zawsze łatwy '
 'trud współpracy w pełnieniu obowiązków, słuchania siebie nawzajem, '


('Na portalu Fronda.pl pojawił się apel o modlitwę za feministkę Katarzynę '
 'Bratkowską. Powód? 16 grudnia w Polsat News działaczka na koniec rozmowy o '
 'planowanym zaostrzeniu kar za aborcję zapowiedziała, że jest w ciąży i '
 'niedługo ją usunie. W rozmowie z NaTemat podała datę: W wigilię, 24 grudnia. '
 'W programie "Tak czy nie" podczas dyskusji o proponowanym przez Komisję '
 'Kodyfikacyjną przy Ministerstwie Sprawiedliwości zaostrzeniu kary za '
 'przerwanie ciąży starli się feministyczna działaczka Katarzyna Bratkowska i '
 'Jacek Żalek, poseł Polski Razem. To mi wygląda na przerażający, ale '
 'prawdziwy tytuł jak z Larssona - "Mężczyźni, którzy nienawidzą kobiet". Nie '
 'jestem w stanie znaleźć żadnego innego uzasadnienia [tego projektu] - mówiła '
 'Bratkowska, komentując proponowane zmiany. "Prawo do życia jest prawem '
 'człowieka" Poseł Żalek podkreślał, że w 21. dniu życia płodu zaczyna mu bić '
 'serce. Przekonywał, że życie ludzkie zaczyna się w momencie poczęcia.

('W piątek rano sąd zdecydował, że były już policjant Witold B., podejrzany o '
 'śmiertelne potrącenie 19-Kasi, zostanie na trzy miesiące zamknięty w '
 'areszcie - Od momentu przyjęcia go do aresztu jest pod opieką psychologa i '
 'wychowawcy. Znajduje się pod szczególną ochroną. Oczywiście naszym zadaniem '
 'jest zapewnienie bezpieczeństwa każdemu osadzonemu, jednak w przypadku '
 'byłego policjanta obowiązują szczególne procedury - mówi "Wyborczej" '
 'Krzysztof Wąsek, zastępca dyrektora Aresztu Śledczego w Lublinie. Witold B. '
 'będzie pilnowany podczas spacerów i prysznica. Na spacerniaku i w łaźni nie '
 'będzie mieszał się z innymi aresztantami. B. do czwartku był policjantem w '
 'komendzie w Łęcznej. W piątek Sąd Rejonowy Lublin-Zachód aresztował go na '
 'trzy miesiące. Mężczyzna przyznał się, że 14 grudnia w Wólce pod Lublinem '
 'wjechał na przechodzącą po pasach 19-letnią Kasię. Zginęła na miejscu, ale '
 'funkcjonariusz nawet nie sprawdził, czy żyje, tylko uciekł. Mech

## Example usage

1. Create API object

In [11]:
bert = BertBaseMultilingualEmbeddingApi()

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


2. Create sentence and pass to feed_forward method

In [12]:
sentence = "Oszczędnością i pracą ludzie się bogacą."
bert.feed_forward(sentence)

3. Generate word/sentence embedding, specify `how`

In [13]:
words_embeddings = bert.create_word_embedding_(how = "sum_last_4")

In [14]:
sent_embedding = bert.create_sentence_embedding_()

4. Visualize tensor

In [15]:
print(sent_embedding)

tensor([ 6.7999e-02, -1.9141e-01,  7.3391e-01,  1.9257e-01,  3.7371e-01,
         4.9427e-01, -2.4747e-01, -1.9059e-01,  2.3879e-02,  5.4254e-02,
         1.4454e-01,  4.9810e-01,  3.0980e-02,  2.1776e-01,  3.4605e-02,
         1.9929e-02,  5.7315e-01, -6.9925e-01,  1.0205e-01,  1.9255e-01,
         1.4388e-01,  7.1202e-02, -2.9616e-03,  1.2551e-01, -9.6333e-02,
         2.8764e-01, -2.8002e-01, -1.3754e-01, -1.2272e-01, -5.4620e-01,
         2.0109e-01,  4.3742e-02, -3.2613e-01,  3.2596e-01, -2.4226e-02,
        -1.1224e-01, -3.0505e-01,  2.9660e-01,  3.9777e-01, -2.0974e-01,
        -3.0772e-01,  2.6053e-01,  2.2329e-01,  1.4007e-01,  2.5690e-01,
        -1.6820e-01,  1.1059e-01, -3.5593e-03, -3.1462e-01,  8.8268e-02,
        -1.5128e-02, -1.2418e-01,  9.7928e-02, -2.7980e-02,  6.9080e-02,
         2.6530e-01,  8.2499e-02, -9.7751e-02,  1.5672e-01,  1.1794e-01,
        -1.2886e-01,  7.4442e-04, -4.3342e-01, -6.0595e-02, -1.7476e-01,
         3.7781e-01, -6.0381e-02, -2.3784e-01,  2.4

In [ ]:
bert.plot_embedding_hist(sent_embedding)

## Visualize embeddings in tensorboard

In [21]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [19]:
LOG_DIR = 'minimalsample'
NAME_TO_VISUALISE_VARIABLE = "example_embeddings"
path_for_mnist_metadata =  'metadata.tsv'

In [ ]:
token_embeddings_np = torch.stack(concatenated_lats_4_layers).numpy()

In [ ]:
embedding_var = tf.Variable(token_embeddings_np, name=NAME_TO_VISUALISE_VARIABLE)
summary_writer = tf.summary.FileWriter(LOG_DIR)

In [20]:
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

# Specify where you find the metadata
embedding.metadata_path = path_for_mnist_metadata #'metadata.tsv'

# Say that you want to visualise the embeddings
projector.visualize_embeddings(summary_writer, config)

NameError: name 'projector' is not defined

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), 1)

In [17]:
with open(path_for_mnist_metadata,'w') as f:
    f.write("Index\tLabel\n")
    for index,label in enumerate(tokenized_text):
        f.write("%d\t %s\n" % (index,label.encode('utf-8')))

NameError: name 'path_for_mnist_metadata' is not defined